In [79]:
import numpy as np
import pandas as pd
import operator
import math

In [61]:
#Define a class for KNN classification
class KNNclassify(object):
    
    def __init__(self,X,Y,norm):   # Initialization，and define whether do standardization
        self.n=X.shape[0]
        self.y=Y
        self.allKNN=[]
        self.allWKNN=[]
        if norm==True:   # Deviation standardization
            self.x=((X-np.tile(np.min(X,axis=0),(self.n,1)))/np.tile(np.max(X,axis=0)-np.min(X,axis=0),(self.n,1)))
        else:
            self.x=X
        
    def DataCheck(self):   
        Count={}
        for i in range(self.n):
            c=self.y[i]
            Count[c]=Count.get(c,0)+1   
        return Count,np.mean(Count.values()),np.var(Count.values())
        
    def Distances(self,test):   # Calculate Euclidean distance
        Diff=np.tile(test,(self.n,1))-self.x
        sqDiff=Diff**2
        sqDistances=sqDiff.sum(axis=1)
        Distances=sqDistances**0.5   
        return Distances        
        
    def KNN(self,test,k):   # kNN classification
        Distances=self.Distances(test)
        sortedDist=Distances.argsort()   # order：return the index of ascending order for finding out the k-nearest neighbors
        Count={}
        for i in range(k):
            vote=self.y[sortedDist[i]]
            Count[vote]=Count.get(vote,0)+1   # Define a dict to count votes
        sortedCount=sorted(Count.items(),key=operator.itemgetter(1),reverse=True)    # Descending order by key value
        self.allKNN.append(sortedCount[0][0])
        return sortedCount[0][0]
    
    def allclass(self,xtest,k):   # Apply kNN classification on the test data set
        ntest=xtest.shape[0]
        allclass=np.zeros(ntest,dtype=str)
        for i in range(ntest):
            allclass[i]=self.KNN(xtest[i],k)
        return allclass
    
    def err(self,xtest,ytest,k):   # Calculate the error rate of kNN
        ntest=xtest.shape[0]
        err=np.sum(ytest!=self.allclass(xtest,k))/float(ntest)
        return err
    
    def weighedKNN(self,test,k):   # Weighted kNN 
        Distances=self.Distances(test)
        sortedDist=Distances.argsort()    # order：return the index of ascending order for finding out the k-nearest neighbors
        Count = {}   # Define a dict to count votes of k-nearest neighbors
        for i in range(k):
            vote=self.y[sortedDist[i]]
            Count[vote]=Count.get(vote,0)+1/Distances[sortedDist[i]]   # Use 1/Distance as weighted vote
        sortedCount = sorted(Count.items(),key=operator.itemgetter(1),reverse=True)   # Descending order by key value
        return sortedCount[0][0]
    
    def allWclass(self,xtest,k):   # Apply weighted kNN classification on the test data set
        ntest=xtest.shape[0]
        allclass=np.zeros(ntest,dtype=str)
        for i in range(ntest):
            allclass[i]=self.weighedKNN(xtest[i],k)
        return allclass
    
    def Werr(self,xtest,ytest,k):   # Calculate the error rate of weighted kNN
        ntest=xtest.shape[0]
        err=np.sum(ytest!=self.allWclass(xtest,k))/float(ntest)
        return err
    
    def allclasses(self,xtest,k):   
        ntest=xtest.shape[0]
        manager=Manager()
        record=[]
        self.allKNN=manager.list()
        for i in range(ntest):
            p=Process(target=self.KNN,args=(xtest[i],k))
            p.daemon=True
            record.append(p)
        for i in range(ntest):
            record[i].start()
        for i in range(ntest):
            record[i].join()
        return self.allKNN

In [62]:
def S(x,y):
    S = [[] for i in range(x.shape[1])]
    for m in range(x.shape[1]):
        V = {}
        for n in range(x.shape[0]):
            v = x[n,m]
            V [v] = V.get(v,0)+1
        for i in V.keys():
            Count = {}
            for n in range(x.shape[0]):
                c=y[n]
                if x[n,m]==i: Count[c] = Count.get(c,0)+1
            for k in Count.keys(): 
                Count[k] = Count.get(k,0)/V[i]
            p = np.fromiter(iter(Count.values()), dtype=float)
            lnp = np.array(list(map(math.log,p)))
            if len(p)==1: 
                s = 0
            else:s = -sum(p*lnp)
            V[i] = s
        S[m] = V
    return S

In [63]:
def Entropy(xtrain,ytrain,test,Sv,k=10):
    D = np.zeros(xtrain.shape[0])
    for n in range(xtrain.shape[0]):
        check = xtrain[n]==test
        if sum(check)==0:
            D[n] = 10
            continue
        d = 0
        for i in range(xtrain.shape[1]):
            if check[i]==True:
                v = xtrain[n,i]
                d = d + Sv[i][v]
        D[n] = d/sum(check)
    #print (D)
    sortedD = D.argsort()
    Count = {}
    for i in range(k):
        c = ytrain[sortedD[i]]
        Count[c]=Count.get(c,0)+1            #Dict
    Kj = Count.items()
    T = np.zeros(len(Kj))
    Dk = D[sortedD[range(k)]]
    C = ytrain[sortedD[range(k)]]
    for j in range(len(Kj)):
        sumD = 0
        for i in range(k):
            if C[i]==list(Kj)[j][0]:
                sumD = sumD+Dk[i]
        a = float(k-list(Kj)[j][1])/float(k)
        t = a*(sumD/list(Kj)[j][1])
        T[j]=t
    C = list(Kj)[T.argsort()[0]][0]
    return C

In [64]:
def AllEntropy(xtrain,ytrain,xtest,k):
    ntest=xtest.shape[0]
    allclass=np.zeros(ntest,dtype=str)
    Sv = S(xtrain,ytrain)
    for i in range(ntest):
        allclass[i]=Entropy(xtrain,ytrain,xtest[i],Sv)   
    return allclass

In [65]:
def Eerr(xtrain,ytrain,xtest,k):
    Allpre = AllEntropy(xtrain,ytrain,xtest,k)
    err=np.sum(ytest!=Allpre)/float(ytest.shape[0])
    return err

In [120]:
x = np.loadtxt("C:/Users/Kiran/Desktop/2017 Fall/Data Mining/Final Project/x.csv",delimiter=',',skiprows=0,dtype=float)
yb = np.loadtxt("C:/Users/Kiran/Desktop/2017 Fall/Data Mining/Final Project/y.csv",delimiter=',',skiprows=0,dtype=bytes)
y = np.loadtxt("C:/Users/Kiran/Desktop/2017 Fall/Data Mining/Final Project/y.csv",delimiter=',',skiprows=0,dtype=str)

In [121]:
N = x.shape[0]
M = x.shape[1]
for n in range(N):
    y[n] = yb[n].decode(encoding='utf-8')
print (y)

['V' 'C' 'J' ..., 'Z' 'Y' 'M']


In [122]:
folds = 10
n = int(N/folds)
print (size)
xtrain = x[:size,]
xtest = x[size:,]
ytrain = y[:size,]
ytest = y[size:,]
print (xtrain,xtest)
print (ytrain,ytest)

2000
[[  3.  10.   5. ...,   9.   0.   8.]
 [  3.   5.   4. ...,   9.   3.   7.]
 [  0.   0.   1. ...,   7.   0.   8.]
 ..., 
 [  8.  15.   8. ...,   9.   3.   7.]
 [  2.   4.   4. ...,   8.   2.   8.]
 [  4.   7.   5. ...,   9.   4.   6.]] [[ 10.  14.  10. ...,  11.   2.   7.]
 [  6.  12.   5. ...,   7.   6.   9.]
 [  6.   9.   6. ...,   8.   6.   7.]
 ..., 
 [  7.  11.   7. ...,   9.   5.   8.]
 [  4.   8.   6. ...,  11.   2.   8.]
 [  3.   2.   5. ...,   5.   2.   8.]]
['V' 'C' 'J' ..., 'K' 'D' 'P'] ['M' 'L' 'R' ..., 'Z' 'Y' 'M']


In [123]:
Sv = S(xtrain,ytrain)
print (Sv)

[{3.0: 3.2204042982523031, 0.0: 0.69314718055994529, 6.0: 3.1781475384092857, 7.0: 3.129313522445015, 4.0: 3.201950361887631, 1.0: 2.8529736858281849, 5.0: 3.2107031983520797, 2.0: 3.1517872995862124, 11.0: 1.6094379124341005, 9.0: 2.4324441210724341, 8.0: 2.8621368130810887, 13.0: 0, 10.0: 2.0431918705451206, 14.0: 0, 12.0: 0}, {10.0: 3.1800743424008364, 5.0: 3.1893108229528107, 0.0: 3.0780271151563996, 6.0: 3.1928966259680465, 11.0: 3.1693186682477528, 3.0: 3.1653018721858439, 8.0: 3.2141941990162137, 9.0: 3.1744108702748712, 4.0: 3.1784747091273644, 15.0: 2.614589432947608, 7.0: 3.2002138227595816, 12.0: 2.8862023141021282, 1.0: 2.9488484168475719, 2.0: 2.9778813320259374, 14.0: 2.1333819302645449, 13.0: 2.8234169983973918}, {5.0: 3.2123469694519127, 4.0: 3.2104957497474622, 1.0: 2.5801308998311092, 6.0: 3.213134144008412, 9.0: 2.8687952490680084, 2.0: 2.9537062638000888, 8.0: 3.0341996619407574, 7.0: 3.1713313311186653, 3.0: 3.062169158108333, 11.0: 1.6769877743224173, 10.0: 1.9288

In [124]:
print (Entropy(xtrain,ytrain,xtest[0,],Sv))
print (ytest[8,])

V
V


In [127]:
result = np.zeros((10,3))
for i in range(0,10):
    NO = np.arange(n)+134*i
    xtrain = x[n*i:n*(i+1),]
    xtest = np.vstack((x[:n*i,],x[n*(i+1):,]))
    ytrain = y[n*i:n*(i+1),]
    ytest = np.hstack((y[:n*i,],y[n*(i+1):,]))
    a=KNNclassify(xtrain,ytrain,norm=False)
    result[i,0]=1-a.err(xtest,ytest,k=10)
    result[i,1]=1-a.Werr(xtest,ytest,k=10)
    result[i,2]=1-Eerr(xtrain,ytrain,xtest,k=10)
print (result)
knn=0
wknn=0
Eknn=0
for i in range(0,10):
    knn=knn+result[i,0]
    wknn=wknn+result[i,1]
    Eknn=Eknn+result[i,2]
print (knn/10,wknn/10,Eknn/10)

[[ 0.73709926  0.77839206  0.84219594]
 [ 0.74290357  0.78419637  0.81732032]
 [ 0.75368301  0.79663418  0.81732032]
 [ 0.76031651  0.79995093  0.80488251]
 [ 0.76695001  0.81155955  0.82809976]
 [ 0.75202463  0.79000068  0.78332364]
 [ 0.7545122   0.79746337  0.80073657]
 [ 0.7611457   0.7949758   0.82644138]
 [ 0.77441269  0.81238874  0.78166526]
 [ 0.7611457   0.80492605  0.79161551]]
0.756419327297 0.797048772086 0.809360121866


In [129]:
print (Entropy(xtrain,ytrain,xtest[0,],Sv))
print (ytest[8,])

V
V


In [151]:
result = np.zeros((10,3))
for s in range(0,10):
    n = (s+1)*100
    knn=0
    wknn=0
    Eknn=0
    for i in range(0,10):
        NO = np.arange(n)+134*i
        xtrain = x[n*i:n*(i+1),]
        xtest = np.vstack((x[:n*i,],x[n*(i+1):,]))
        ytrain = y[n*i:n*(i+1),]
        ytest = np.hstack((y[:n*i,],y[n*(i+1):,]))
        a=KNNclassify(xtrain,ytrain,norm=False)
        knn = knn+1-a.err(xtest,ytest,k=10)
        wknn = wknn+1-a.Werr(xtest,ytest,k=10)
        Eknn = Eknn+1-Eerr(xtrain,ytrain,xtest,k=10)
    result[s,] = np.array([knn/10,wknn/10,Eknn/10])
print (result)

[[ 0.23818444  0.33835102  0.55786443]
 [ 0.34145965  0.45204741  0.64676617]
 [ 0.41316759  0.50475571  0.69216458]
 [ 0.47965148  0.55576227  0.69965241]
 [ 0.52299326  0.59869321  0.72942815]
 [ 0.58971674  0.65990183  0.74736266]
 [ 0.61203422  0.67973679  0.76421799]
 [ 0.64355582  0.69911503  0.77648753]
 [ 0.65320119  0.71437475  0.78368476]
 [ 0.66815671  0.73324071  0.77597493]]


In [157]:
result = np.zeros((10,3))
n = 200
for p in range(0,10):
    k = (p+1)*5
    knn=0
    wknn=0
    Eknn=0
    for i in range(0,10):
        NO = np.arange(n)+134*i
        xtrain = x[n*i:n*(i+1),]
        xtest = np.vstack((x[:n*i,],x[n*(i+1):,]))
        ytrain = y[n*i:n*(i+1),]
        ytest = np.hstack((y[:n*i,],y[n*(i+1):,]))
        a=KNNclassify(xtrain,ytrain,norm=False)
        knn = knn+1-a.err(xtest,ytest,k=10)
        wknn = wknn+1-a.Werr(xtest,ytest,k=10)
        Eknn = Eknn+1-Eerr(xtrain,ytrain,xtest,k=10)
    result[p,] = np.array([knn/10,wknn/10,Eknn/10])
print (result)

[[ 0.4353249   0.4885045   0.69643067]
 [ 0.34145965  0.45204741  0.64676617]
 [ 0.30560835  0.42727395  0.62179354]
 [ 0.323636    0.43560317  0.63879441]
 [ 0.28531252  0.42372297  0.61551394]
 [ 0.24825977  0.38371185  0.60857102]
 [ 0.2508424   0.30097481  0.59322387]
 [ 0.26763145  0.36121686  0.60052229]
 [ 0.25959786  0.36819351  0.5811371 ]
 [ 0.25629965  0.35384866  0.58486375]]


In [158]:
data = np.loadtxt('C:/Users/Kiran/Desktop/2017 Fall/Data Mining/Final Project/Data-NBA/NBA-leveled data.csv',
                  delimiter=',',skiprows=1,dtype=int)
print (data)

[[   1    3    6 ...,    1    2    0]
 [   2    3    6 ...,    1    3    0]
 [   3    8    3 ...,    0    2    0]
 ..., 
 [1338    4    2 ...,    1    1    0]
 [1339    5    2 ...,    0    1    1]
 [1340    5    2 ...,    0    1    1]]


In [159]:
x = data[:,1:-1]
y = np.array(data[:,-1],dtype=str)
print (x,y)

[[3 6 2 ..., 1 1 2]
 [3 6 2 ..., 4 1 3]
 [8 3 1 ..., 1 0 2]
 ..., 
 [4 2 1 ..., 1 1 1]
 [5 2 1 ..., 1 0 1]
 [5 2 1 ..., 2 0 1]] ['0' '0' '0' ..., '0' '1' '1']


In [160]:
N = y.shape[0]
s = 10
n = int(N/s)
xtrain = x[:n,]
xtest = x[n:,]
ytrain = y[:n,]
ytest = y[n:,]
print (xtrain,xtest)
print (ytrain,ytest)

[[3 6 2 ..., 1 1 2]
 [3 6 2 ..., 4 1 3]
 [8 3 1 ..., 1 0 2]
 ..., 
 [5 1 1 ..., 0 1 0]
 [9 4 3 ..., 3 1 3]
 [9 4 2 ..., 1 1 3]] [[ 9  4  3 ...,  2 -1  3]
 [ 8  4  2 ...,  2  0  1]
 [ 9  4  3 ...,  2  1  3]
 ..., 
 [ 4  2  1 ...,  1  1  1]
 [ 5  2  1 ...,  1  0  1]
 [ 5  2  1 ...,  2  0  1]]
['0' '0' '0' '1' '1' '0' '1' '1' '0' '0' '0' '1' '1' '0' '1' '1' '0' '0'
 '0' '1' '1' '1' '1' '0' '0' '1' '0' '1' '0' '1' '1' '1' '1' '1' '0' '1'
 '1' '1' '1' '1' '0' '0' '0' '0' '1' '1' '1' '0' '1' '1' '0' '1' '1' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0' '1' '0' '1' '1' '1' '1' '0'
 '1' '0' '1' '1' '1' '1' '0' '1' '1' '0' '0' '1' '0' '0' '1' '1' '0' '1'
 '0' '1' '0' '0' '0' '1' '1' '1' '0' '1' '0' '0' '1' '0' '0' '1' '0' '1'
 '0' '0' '1' '1' '1' '0' '1' '1' '0' '1' '1' '0' '0' '1' '1' '1' '1' '1'
 '1' '0' '1' '1' '1' '0' '1' '1'] ['1' '0' '1' ..., '0' '1' '1']


In [161]:
for m in range(10):
    Count = {}    
    for i in range(x.shape[0]):
        c=y[i]
        if x[i,0]==m: Count[c]=Count.get(c,0)+1   #Dict
        #else: Count[c]=Count.get(c,0)
    v = sum(Count.values())
    if v == 0: continue
    if len(Count) == 1: 
        S[m,n] = 0
        continue
    print (np.array(list(map(float,Count.values())))/v)

[ 0.8  0.2]
[ 0.81818182  0.18181818]
[ 0.3125  0.6875]
[ 0.6302521  0.3697479]
[ 0.56589147  0.43410853]
[ 0.49056604  0.50943396]
[ 0.56834532  0.43165468]
[ 0.6516129  0.3483871]
[ 0.28965517  0.71034483]
[ 0.1558753  0.8441247]


In [162]:
Sv = S(xtrain,ytrain)
print (Sv)

[{3: 0.63651416829481278, 8: 0.65548177390139273, 6: 0.68900923847665863, 5: 0.69314718055994529, 9: 0.53316384073729861, 7: 0.67301166700925652, 4: 0.69314718055994529, 2: 0.63651416829481278, 1: 0, 0: 0}, {6: 0.63651416829481278, 3: 0.67749440444870723, 2: 0.69234696708996146, 1: 0.67000897982742713, 0: 0.69092330931381807, 9: 0.63651416829481278, 7: 0.52970619905765448, 5: 0.47413931305783735, 4: 0.51957983913051542, 8: 0.63651416829481278}, {2: 0.64503315167603414, 1: 0.69314718055994529, 0: 0.65405481030693169, 6: 0.56233514461880829, 4: 0.6108643020548935, 3: 0.45056120886630463, 5: 0.63651416829481278, 8: 0}, {2: 0.50040242353818787, 1: 0.410116318288409, 3: 0.69189665920507992, 5: 0.36692499127270961, 4: 0.64744663903463251, 6: 0}, {2: 0.68490624041583559, -1: 0.63100519280469347, 3: 0.64924835487089805, 4: 0.67301166700925652, 1: 0.66156323815798213, 0: 0.67301166700925652, 9: 0}, {6: 0.68062948789166344, 7: 0.6896709283570267, 8: 0.67590103057401141, 2: 0, 4: 0.63651416829481

In [163]:
print (Entropy(xtrain,ytrain,xtest[0,:],Sv,k=10))
print (ytest[0,])

1
1


In [164]:
k=10
pre = AllEntropy(xtrain,ytrain,xtest,k)
print (pre)

['1' '1' '1' ..., '1' '0' '0']


In [165]:
k=10
err = Eerr(xtrain,ytrain,xtest,k)
print (err)

0.328358208955


In [166]:
N = y.shape[0]
s = 10
n = int(N/s)
k = 10
print (N,n)

1340 134


In [119]:
result = np.zeros((10,3))
for i in range(0,10):
    NO = np.arange(n)+134*i
    xtrain = x[n*i:n*(i+1),]
    xtest = np.vstack((x[:n*i,],x[n*(i+1):,]))
    ytrain = y[n*i:n*(i+1),]
    ytest = np.hstack((y[:n*i,],y[n*(i+1):,]))
    a=KNNclassify(xtrain,ytrain,norm=False)
    result[i,0]=1-a.err(xtest,ytest,k=10)
    result[i,1]=1-a.Werr(xtest,ytest,k=10)
    result[i,2]=1-Eerr(xtrain,ytrain,xtest,k=10)
print (result)
knn=0
wknn=0
Eknn=0
for i in range(0,10):
    knn=knn+result[i,0]
    wknn=wknn+result[i,1]
    Eknn=Eknn+result[i,2]
print (knn/10,wknn/10,Eknn/10)

[[ 0.64510779  0.70999694  0.95811763]
 [ 0.65091211  0.71580125  0.93324201]
 [ 0.66169154  0.72823906  0.93324201]
 [ 0.66832504  0.73155581  0.9208042 ]
 [ 0.67495854  0.74316443  0.94402145]
 [ 0.66003317  0.72160556  0.89924533]
 [ 0.66252073  0.72906825  0.91665826]
 [ 0.66915423  0.72658069  0.94236307]
 [ 0.68242123  0.74399362  0.89758695]
 [ 0.66915423  0.73653094  0.9075372 ]]
0.664427860697 0.728653654946 0.925281813906
